In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sub = pd.read_csv('../data/sample_submission.csv')
age = pd.read_csv('../data/age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

In [3]:
train.shape, test.shape

((2952, 15), (1022, 14))

### 데이터에 대한 간략한 정보 및 결측치 확인

In [7]:
train.info(), train.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

(None,
 단지코드                              0
 총세대수                              0
 임대건물구분                            0
 지역                                0
 공급유형                              0
 전용면적                              0
 전용면적별세대수                          0
 공가수                               0
 자격유형                              0
 임대보증금                           569
 임대료                             569
 도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
 도보 10분거리 내 버스정류장 수                4
 단지내주차면수                           0
 등록차량수                             0
 dtype: int64)

In [8]:
test.info(), test.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   object 
 3   지역                            1022 non-null   object 
 4   공급유형                          1022 non-null   object 
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1020 non-null   object 
 9   임대보증금                         842 non-null    object 
 10  임대료                           842 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  980 non-null    float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

(None,
 단지코드                              0
 총세대수                              0
 임대건물구분                            0
 지역                                0
 공급유형                              0
 전용면적                              0
 전용면적별세대수                          0
 공가수                               0
 자격유형                              2
 임대보증금                           180
 임대료                             180
 도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
 도보 10분거리 내 버스정류장 수                0
 단지내주차면수                           0
 dtype: int64)

### 컬럼명 변경

In [9]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', 
                 '전용면적', '전용면적별세대수', '공가수', '신분', '임대보증금', 
                 '임대료', '지하철', '버스', '단지내주차면수', '등록차량수']

test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', 
                '전용면적', '전용면적별세대수', '공가수', '신분', '임대보증금', 
                '임대료', '지하철', '버스', '단지내주차면수']

### 지역명을 숫자로 맵핑 : 라벨인코딩 1

In [11]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
local_map

{'경상북도': 0,
 '경상남도': 1,
 '대전광역시': 2,
 '경기도': 3,
 '전라북도': 4,
 '강원도': 5,
 '광주광역시': 6,
 '충청남도': 7,
 '부산광역시': 8,
 '제주특별자치도': 9,
 '울산광역시': 10,
 '충청북도': 11,
 '전라남도': 12,
 '대구광역시': 13,
 '서울특별시': 14,
 '세종특별자치시': 15}

In [12]:
train['지역'] = train['지역'].map(local_map)
test['지역'] = test['지역'].map(local_map)

### 전용면적을 5의 배수로 변경

In [13]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

In [16]:
train.loc[:,['전용면적']]

,전용면적
0,35.0
1,35.0
2,50.0
3,50.0
4,50.0
...,...
2947,45.0
2948,50.0
2949,50.0
2950,50.0


### 전용면적 상/하한 적용 - 상한100, 하한 15

In [17]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [18]:
test['전용면적'].unique()

array([ 35.,  45.,  50.,  30.,  55.,  25.,  75., 100.,  15.,  20.,  40.,
        60.,  80.,  70.])

### 단지별 데이터 1차원으로 취합

In [20]:
columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [22]:
area_columns

['면적_35.0',
 '면적_50.0',
 '면적_55.0',
 '면적_30.0',
 '면적_45.0',
 '면적_40.0',
 '면적_25.0',
 '면적_70.0',
 '면적_15.0',
 '면적_20.0',
 '면적_100.0',
 '면적_60.0',
 '면적_75.0',
 '면적_80.0',
 '면적_65.0']

In [23]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [24]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

423it [00:09, 43.61it/s]
150it [00:03, 45.44it/s]


In [25]:
new_train

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_35.0,면적_50.0,면적_55.0,...,면적_25.0,면적_70.0,면적_15.0,면적_20.0,면적_100.0,면적_60.0,면적_75.0,면적_80.0,면적_65.0,등록차량수
0,C2483,900.0,38.0,0.0,1425.0,0.0,3.0,149.0,665.0,86.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.0
1,C2515,545.0,17.0,1.0,624.0,0.0,3.0,80.0,132.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0
2,C1407,1216.0,13.0,2.0,1285.0,1.0,1.0,0.0,124.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1064.0
3,C1945,755.0,6.0,3.0,734.0,1.0,3.0,240.0,303.0,212.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730.0
4,C1470,696.0,14.0,4.0,645.0,0.0,2.0,254.0,246.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,C2586,90.0,7.0,9.0,66.0,0.0,3.0,36.0,0.0,0.0,...,42.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0
419,C2035,492.0,24.0,5.0,521.0,0.0,1.0,156.0,0.0,0.0,...,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,246.0
420,C2020,40.0,7.0,8.0,25.0,1.0,2.0,15.0,0.0,0.0,...,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
421,C2437,90.0,12.0,11.0,30.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,16.0


In [26]:
new_train.isna().sum()

단지코드         0
총세대수         0
공가수          0
지역           0
단지내주차면수      0
지하철         20
버스           1
면적_35.0      0
면적_50.0      0
면적_55.0      0
면적_30.0      0
면적_45.0      0
면적_40.0      0
면적_25.0      0
면적_70.0      0
면적_15.0      0
면적_20.0      0
면적_100.0     0
면적_60.0      0
면적_75.0      0
면적_80.0      0
면적_65.0      0
등록차량수        0
dtype: int64

### 결측치 처리

In [27]:
new_train = new_train.fillna(-1)
new_test = new_test.fillna(-1)

In [29]:
new_train.iloc[:, 1:-1]

,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_35.0,면적_50.0,면적_55.0,면적_30.0,...,면적_40.0,면적_25.0,면적_70.0,면적_15.0,면적_20.0,면적_100.0,면적_60.0,면적_75.0,면적_80.0,면적_65.0
0,900.0,38.0,0.0,1425.0,0.0,3.0,149.0,665.0,86.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,545.0,17.0,1.0,624.0,0.0,3.0,80.0,132.0,0.0,276.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1216.0,13.0,2.0,1285.0,1.0,1.0,0.0,124.0,0.0,390.0,...,362.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,755.0,6.0,3.0,734.0,1.0,3.0,240.0,303.0,212.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,696.0,14.0,4.0,645.0,0.0,2.0,254.0,246.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,90.0,7.0,9.0,66.0,0.0,3.0,36.0,0.0,0.0,0.0,...,0.0,42.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
419,492.0,24.0,5.0,521.0,0.0,1.0,156.0,0.0,0.0,0.0,...,0.0,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
420,40.0,7.0,8.0,25.0,1.0,2.0,15.0,0.0,0.0,0.0,...,0.0,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
421,90.0,12.0,11.0,30.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0


In [31]:
new_train.iloc[:,-1]

0      1015.0
1       205.0
2      1064.0
3       730.0
4       553.0
        ...  
418      57.0
419     246.0
420      19.0
421      16.0
422     146.0
Name: 등록차량수, Length: 423, dtype: float64

### 학습

In [28]:
from sklearn.ensemble import RandomForestRegressor

In [32]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [34]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(x_train, y_train)

pred = model.predict(x_test)
pred

array([ 694.89, 1182.82,  481.82,  545.32, 1047.36, 1680.1 ,  931.24,
        502.88,  389.4 ,  259.41,  436.14,  252.31,  439.06,  255.69,
        231.23,  353.32,  434.64,  284.44,  179.96,  687.1 ,  252.17,
        560.03,  582.3 ,  513.75,  446.69,  147.24,  128.05,  611.86,
        460.11,  553.84,  912.81,  143.17,  379.26,  232.3 ,   87.07,
        287.8 ,  409.37,  558.68,  781.38,  318.28,  455.1 ,  541.59,
        501.78,  504.25,  751.01, 1138.42,  460.44,  587.2 ,  401.79,
        380.84,  721.51,  288.37, 1152.9 ,  683.14,  662.62,  307.38,
        499.78,  273.89,  470.22,   88.41,  384.79,  598.58, 1002.19,
        496.45,   86.9 ,  278.27,  668.51, 1117.12,  531.55,  415.95,
        676.9 ,  292.93,  588.75,  729.41,  792.91,  486.85,  682.73,
        884.4 ,  742.21,  763.18,  626.46, 1161.06,  338.38,  250.12,
        330.93,  222.07,  255.85,  424.02,  253.24,  994.55,  930.56,
        665.59,  303.25,  510.77, 1129.56,  917.82,  595.67, 1045.22,
        860.71,  814

In [35]:
sub.columns

Index(['code', 'num'], dtype='object')

In [36]:
sub['num'] = pred

In [38]:
sub.to_csv('baseline_by_dacon.csv', index=False)

In [ ]:
### 125.0751020408